In [ ]:
import json

#recursive fn for iterting through each structre.
def get_val_json(data,keys):
  if len(keys) == 1:
    return data.get(keys[0])
  else:
    new_key = keys[0]
    if new_key in data:
      return get_val_json(data[new_key],keys[1:])
    else:
      return None

# Load JSON data from the file
with open('alogparams_from_ui1.json', 'r') as json_file:
    j_data = json.load(json_file)

#keys to be extracted
t_type_keys = ['design_state_data', 'target', 'type']
t_name_keys = ['design_state_data', 'target', 'target']

#extract values using the generic function above
type_val = get_val_json(data,t_type_keys)
target_val = get_val_json(data,t_name_keys)

print("Type:", type_val)
print("Target:", target_val)

In [ ]:
#2.importing datasets and Reading the features 
#.. what missing imputation needs to be applied

import pandas as pd
iris_df = pd.read_csv('iris.csv')
# print(iris_df.head())

#extracting feature handling metrics from json ->j_data
feat_hdg_info = j_data['design_state_data']['feature_handling']

#loooing through features and applying missing value imputaions
for feature_name, feature_details in feat_hdg_info.items():
  if feature_details['feature_details']['missing_values'] == 'Impute':
    impute_method = feature_details['feature_details']['impute_with']
    impute_value = feature_details['feature_details']['impute_value']
    #apply imputation
    if impute_method =='Average of values':
      iris_df[feature_name].fillna(iris_df[feature_name].mean(), inplace=True)
    elif impute_method == 'custom':
      iris_df[feature_name].fillna(impute_value, inplace=True)